## Test CaddDataset

In [ ]:
from gin_train.trainers import KerasTrainer

In [ ]:
from kipoi_cadd.data import CaddDataset
from kipoi_cadd.data import cadd_train_valid_data
from kipoi_cadd.data import save_variant_ids
from kipoi_cadd.data import train_test_split_indexes
from kipoi_cadd.config import get_data_dir
from random import shuffle
from kipoi_cadd.models import logistic_regression_scikit, logistic_regression_keras
from gin_train.metrics import ClassificationMetrics
from sklearn.model_selection import train_test_split
%load_ext line_profiler

## 1. Check the dataset class

In [3]:
variant_id_file = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/sample_variant_ids.pkl"
lmdb_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/lmdb/"
ds = CaddDataset(lmdb_dir, variant_id_file)
print("Version", ds.version)
ds.__getitem__(234)

Version 1.3


{'inputs': array([0., 1., 0., ..., 0., 0., 0.], dtype=float16),
 'targets': 0,
 'metadata': {'variant_id': "1:404971:C:['T']"}}

In [4]:
set_10000 = list(range(10000))
shuffle(set_10000)

In [5]:
%%time
for var in set_10000:
    ds.__getitem__(var)

CPU times: user 1.3 s, sys: 77.8 ms, total: 1.37 s
Wall time: 1.37 s


In [6]:
%lprun -f ds.__getitem__ for var in range(512): ds.__getitem__(var)

Timer unit: 1e-06 s

Total time: 0.072957 s
File: /data/ouga04b/ag_gagneur/project_local/kipoi-cadd/kipoi_cadd/data.py
Function: __getitem__ at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                               def __getitem__(self, idx):
    39       512        604.0      1.2      0.8          if self.lmdb_cadd is None:
    40                                                       self.lmdb_cadd = lmdb.Environment(
    41                                                           self.lmdb_cadd_path, map_size=self._map_size, lock=False)
    42                                           
    43                                                   # TODO: Make this distiction clearer, do we want to search by loc or iloc?
    44                                                   # Loc breaks when a splitted dataset doesn't have idx.
    45                                                   # variant_id = self.variant_ids.loc[idx]
    46  

In [7]:
%%time
# Create train, test and validation indexes
all_variant_id_file = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/variant_ids.pkl"
training_data_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
train_idx_path = training_data_dir + "train_idx.pkl"
test_idx_path = training_data_dir + "test_idx.pkl"
valid_idx_path = training_data_dir + "valid_idx.pkl"
lmdb_dir = training_data_dir + "lmdb"
"""
train_idx, test_idx = train_test_split_indexes(all_variant_id_file, test_size=0.4)
save_variant_ids(test_idx_path, test_idx)
test_idx, valid_idx = train_test_split_indexes(test_idx_path, test_size=0.5)

save_variant_ids(train_idx_path, train_idx)
save_variant_ids(test_idx_path, test_idx)
save_variant_ids(valid_idx_path, test_idx)
"""
train, valid = cadd_train_valid_data(lmdb_dir, train_idx_path, valid_idx_path)

CPU times: user 4.92 s, sys: 2.98 s, total: 7.9 s
Wall time: 7.89 s


In [10]:
%%time
train_it = train.batch_train_iter(batch_size=512, num_workers=10, shuffle=True)

CPU times: user 222 µs, sys: 0 ns, total: 222 µs
Wall time: 232 µs


## TODO

- can we have multiple connections to LMDB open at the same time?
  - just open two lmdb's in the same python or different python session

In [13]:
%%time
ds = CaddDataset(lmdb_dir, train_idx_path)
ds_iter = ds.batch_train_iter(batch_size=512, num_workers=1, shuffle=True)
next(ds_iter)

CPU times: user 5.47 s, sys: 4.42 s, total: 9.9 s
Wall time: 9.88 s


## 2. Train a logistic regression model

In [18]:
tr = KerasTrainer(logistic_regression_keras(n_features=1063), train, valid, get_data_dir() + "/models/try10")
tr.train(batch_size=512, epochs=10, num_workers=10)

Epoch 1/10
 5888/41066 [===>..........................] - ETA: 48:03 - loss: nan - acc: 0.0030

Process Process-31:
Process Process-30:
Traceback (most recent call last):
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd2/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd2/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd2/lib/python3.6/site-packages/kipoi/external/torch/data.py", line 58, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd2/lib/python3.6/site-packages/kipoi/external/torch/data.py", line 58, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/data/ouga04b/ag_gagneur/project_local/kipoi-cadd/kipoi_cadd/data.py", line 50, in __getitem__
    item = pa.deserialize(buf)
KeyboardInterrupt
Traceback (most recent call last):
  File "/opt/modules/

KeyboardInterrupt: 

In [10]:
metric = ClassificationMetrics()
tr.evaluate(metric)

8it [00:00, 46.16it/s]                       
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd2/lib/python3.6/site-packages/sklearn/metrics/ranking.py:521: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


OrderedDict([('auPR', nan),
             ('auROC', nan),
             ('accuracy', 1.0),
             ('n_positive', 0),
             ('n_negative', 2000),
             ('frac_positive', 0.0)])

In [16]:
# training
train.batch_size = 256  # 512
train.num_workers = 10

# train-specific
train.epochs = 200
train.early_stop_patience = 4
train.train_epoch_frac = 1.0  # optionally subset the training set
train.valid_epoch_frac = 1.0  # optionally subset the validation set
